In [1]:
from pydantic import BaseModel
from sqlalchemy import create_engine
import pandas as pd
from google.cloud import bigquery




class UserTravelDetails(BaseModel):
    country: str
    max_budget: float
    min_budget: float
    departing_after: str
    departing_before: str
    max_duration: int
    min_duration: int

def big_query_filter(user_travel_details: UserTravelDetails):

    PROJECT = "wagon-bootcamp-377120"
    DATASET = "g_adventures_dataset"
    TABLE = "one_month"
    
    query = f"SELECT DISTINCT tour_name FROM {PROJECT}.{DATASET}.{TABLE} WHERE 1 = 1"

    # Iterate through the provided filter criteria and add them to the query
    if user_travel_details.country:
        query += f" AND visited_countries LIKE '%{user_travel_details.country}%'"

    if user_travel_details.max_budget:
        query += f" AND Standard___Adult <= {user_travel_details.max_budget}"

    if user_travel_details.min_budget:
        query += f" AND Standard___Adult >= {user_travel_details.min_budget}"

    if user_travel_details.departing_after:
        query += f" AND start_date >= '{user_travel_details.departing_after}'"

    if user_travel_details.departing_before:
        query += f" AND start_date <= '{user_travel_details.departing_before}'"

    if user_travel_details.max_duration:
        query += f" AND duration <= {user_travel_details.max_duration}"

    if user_travel_details.min_duration:
        query += f" AND duration >= {user_travel_details.min_duration}"
        
    client = bigquery.Client(project=PROJECT)
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()

    return list(df["tour_name"])





In [2]:
# Example usage:
user_travel_details = UserTravelDetails(
    country="Argentina",
    max_budget=2000,
    min_budget=0,
    departing_after="2024-04-15",
    departing_before="2024-04-30",
    max_duration=10,
    min_duration=7
)

In [3]:
df = big_query_filter(user_travel_details)

In [4]:
df

['Highlights of Chile & Argentina', 'Iguassu & Beyond']